In [4]:
## Instalaciones

%pip install torch
%pip install open3d


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [36]:
## Dependencias

import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import open3d as o3d
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("usando", DEVICE)

usando cpu


In [37]:
## T-net
"""
T-net es una 'mini-red' que aprende una matriz de transformación de tamaño
dimxdim que transforma la entrada a una representación 'canónica', la cuál
es invariante a transformaciones rigidas (rotación, translación, reflexión).
"""

class Tnet(nn.Module):
    def __init__(self, dim, num_points):
        super(Tnet, self).__init__()

        self.dim = dim

        # Función de activación
        self.act = F.relu

        # Conv1d es una implementación sencilla de una 'MLP compartida'
        self.shared_mlp1 = nn.Conv1d(dim, 64, kernel_size=1)
        self.shared_mlp2 = nn.Conv1d(64, 128, kernel_size=1)
        self.shared_mlp3 = nn.Conv1d(128, 1024, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

        self.max_pool = nn.MaxPool1d(kernel_size=num_points)

        # MLPs no compartidas
        self.linear1 = nn.Linear(1024, 512)
        self.linear2 = nn.Linear(512, 256)
        self.linear3 = nn.Linear(256, dim**2)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)
    
    def forward(self, x):
        bs = x.shape[0]

        # Paso a través de las MLPs compartidas
        x = self.bn1(self.act(self.shared_mlp1(x)))
        x = self.bn2(self.act(self.shared_mlp2(x)))
        x = self.bn3(self.act(self.shared_mlp3(x)))

        # Max pool
        x = self.max_pool(x).view(bs, -1)
        
        # Paso a través de las MLPs no compartidas
        x = self.bn4(self.act(self.linear1(x)))
        x = self.bn5(self.act(self.linear2(x)))
        x = self.linear3(x)
        
        # Reshape de 'T-Net(x)' a una matriz
        x = x.view(-1, self.dim, self.dim)
        # Le sumamos la matriz identidad para mayor estabilidad
        iden = torch.eye(self.dim, requires_grad=True).repeat(bs, 1, 1)
        if x.is_cuda:
            iden = iden.cuda()
        x += iden

        return x

In [38]:
## Point-net classifier

class PointnetClassifier(nn.Module):
    def __init__(self, dim, num_points, num_global_feats, num_classes):
        super(PointnetClassifier, self).__init__()

        # Función de activación
        self.act = F.relu

        # T-Net en los puntos de la entrada
        self.input_transform = Tnet(dim, num_points)

        # Primera MLP compartida, transforma los puntos de la entrada en features
        self.shared_mlp1 = nn.Conv1d(3, 64, kernel_size=1)
        self.shared_mlp2 = nn.Conv1d(64, 64, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(64)

        # T-Net en las features
        self.feature_transform = Tnet(64, num_points)

        # Segunda MLP compartida, determina las features globales
        self.shared_mlp3 = nn.Conv1d(64, 64, kernel_size=1)
        self.shared_mlp4 = nn.Conv1d(64, 128, kernel_size=1)
        self.shared_mlp5 = nn.Conv1d(128, num_global_feats, kernel_size=1)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(128)
        self.bn5 = nn.BatchNorm1d(num_global_feats)
        # Max pool para extraer las features globales
        # Devolver los indices nos permite ver los indices críticos que determinan las features globales
        self.max_pool = nn.MaxPool1d(kernel_size=num_points, return_indices=True)

        # MLP para clasificación
        self.linear1 = nn.Linear(num_global_feats, 512)
        self.linear2 = nn.Linear(512, 256)
        self.bn_linear1 = nn.BatchNorm1d(512)
        self.bn_linear2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)

        # Output layer
        self.linear3 = nn.Linear(256, num_classes)
    
    def forward(self, x):

        # Tamaño del batch, es decir cuantos ejemplos hay en el batch
        bs = x.shape[0]

        # Transformación del input
        input_matrix = self.input_transform(x)
        # x = torch.bmm(x.tranpose(2, 1), input_matrix).tranpose(2, 1)
        x = torch.transpose(torch.bmm(torch.transpose(x, 2, 1), input_matrix), 2, 1)

        # Paso a través de las primeras MLPs compartidas
        x = self.bn1(self.act(self.shared_mlp1(x)))
        x = self.bn2(self.act(self.shared_mlp2(x)))

        # Transformación de features
        feature_matrix = self.feature_transform(x)
        # x = torch.bmm(x.tranpose(2, 1), feature_matrix).tranpose(2, 1)
        x = torch.transpose(torch.bmm(torch.transpose(x, 2, 1), feature_matrix), 2, 1)

        # Paso a través de las segundas MLPs compartidas
        x = self.bn3(self.act(self.shared_mlp3(x)))
        x = self.bn4(self.act(self.shared_mlp4(x)))
        x = self.bn5(self.act(self.shared_mlp5(x)))

        global_features, critical_indexes = self.max_pool(x)
        global_features = global_features.view(bs, -1)
        critical_indexes = critical_indexes.view(bs, -1)

        # Clasificación
        x = self.bn_linear1(self.act(self.linear1(global_features)))
        x = self.bn_linear2(self.act(self.linear2(x)))
        x = self.dropout(x)
        x = self.linear3(x)

        # Devolver logits
        return x, critical_indexes, feature_matrix

            

In [65]:
# Clase para el cómputo de la pérdida

class PointNetLoss(nn.Module):
    """
    Atributos:
        alpha            El peso de las clases para pérdida CrossEntropy.
        reg_weight       Peso de regularización.
        size_average     Booleano que define si es que la pérdida final se computa como promedio o no.
    """
    def __init__(self, alpha=None, gamma=0, reg_weight=0, size_average=True):
        super(PointNetLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reg_weight = reg_weight
        self.size_average = size_average

        # Convertimos en tensores
        if isinstance(alpha, (float, int)):
            self.alpha = torch.Tensor([alpha, 1 - alpha])
        if isinstance(alpha, (list, np.ndarray)):
            self.alpha = torch.Tensor(alpha)

        self.cross_entropy_loss = nn.CrossEntropyLoss(weight=self.alpha)

    def forward(self, predictions, targets, A):
        # tamaño de batch
        batch_size = predictions.size(0)

        # computamos pérdida CE
        ce_loss = self.cross_entropy_loss(predictions, targets)
        
        # Probabilidades predichas
        pn = F.softmax(predictions, dim=1)
        pn.gather(1, targets.view(-1, 1)).view(-1)

        if self.reg_weight > 0:
            I = torch.eye(64).unsqueeze(0).repeat(A.shape[0], 1, 1)
            if A.is_cuda:
                I = I.cuda()
            reg = torch.linalg.norm(I - torch.bmm(A, A.transpose(2, 1)))
            reg = self.reg_weight * reg / batch_size

        focal_loss = ((1 - pn)**self.gamma * ce_loss)
        if self.size_average:
            return focal_loss.mean() + reg
        else:
            return focal_loss.sum() + reg

In [66]:
class ModelNet(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return  len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [67]:
ROOT_DIR = os.getcwd()
DATASET_DIR = os.path.join(ROOT_DIR, "ModelNet10")
classes = {
    "toilet": 0,
    "monitor": 1
}



x_train = list()
y_train = list()

for label, value in classes.items():
    TRAIN_CLASS_DIR = os.path.join(DATASET_DIR, label, "train")  
    
    for file in os.scandir(TRAIN_CLASS_DIR):  
        if ".pcd" in str(file):
            pcd = o3d.io.read_point_cloud(file.path)
            points = np.asarray(pcd.points, dtype=float)
            x_train.append(points)
            y_train.append(value)


x_train = np.transpose(x_train, (0, 2, 1))

train_data = ModelNet(x_train, y_train)

In [69]:
# parametros del dataset
batch_size = 16
dim = x_train.shape[1]
num_points = 1024
num_classes = 2

# hiperparametros
num_global_feats = 1024     # número de features globales calculadas
epochs = 200
learning_rate = 0.01
reg_weight = 0.001



train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes).to(DEVICE)

optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)
criterion = PointNetLoss(alpha=None, gamma=1, reg_weight=reg_weight, size_average=True).to(DEVICE)

loss_dict = {
    "train": {
        "loss": list(),
        "acc": list()
    },
    "val_loss": {
        "loss": list(),
        "acc": list()
    }
}

for epoch in range(1, epochs + 1):
    # Modo entrenamiento
    classifier = classifier.train()

    batch_losses = list()
    batch_accs = list()
    
    for pcds, labels in train_loader:
        pcds = pcds.to(DEVICE)
        labels = labels.squeeze().to(DEVICE)

        # Gradientes en cero
        optimizer.zero_grad()
        
        # Hacer predicciones
        out, _, A = classifier(pcds)
        loss = criterion(out, labels, A)
                
        # Calcular gradiente y optimizar
        loss.backward()
        optimizer.step()

        # Calculamos las elecciones
        pred_choice = torch.softmax(out, dim=1).argmax(dim=1)
        
        # Elecciones correctas
        correct = pred_choice.eq(labels.data).cpu().sum()
        accuracy = correct.item() / float(pcds.size(0))

        # Registramos métricas para el batch
        batch_losses.append(loss.item() * pcds.size(0))
        batch_accs.append(accuracy)
        
    epoch_train_loss = np.mean(batch_losses)
    epoch_train_acc = np.mean(batch_accs)
    
    print("Loss", epoch_train_loss)
    print("Acc", epoch_train_acc)
    loss_dict["train"]["loss"].append(epoch_train_loss)
    loss_dict["train"]["acc"].append(epoch_train_loss)


Loss 18.674128275291594
Acc 0.5550108932461874


KeyboardInterrupt: 

In [ ]:


# parametros del dataset
batch_size = 32             # número de point clouds
dim = 3                     # número de dimensiones por cada punto
num_points = 1024           # número de puntos por point cloud
num_classes = 2             # número de clases de clasificación

# hiperparametros
num_global_feats = 1024     # número de features globales calculadas

test_data = torch.rand(batch_size, dim, num_points)
print(test_data.shape)

classifier = PointnetClassifier(dim, num_points, num_global_feats, num_classes)
out, _, _ = classifier(test_data)
print(f'Class output shape: {out.shape}')
print(f'Class output: {out}')

train_test(50, 0.01, 0.001, num_classes)